In [ ]:
# !pip install python-frontmatter

In [1]:
import frontmatter
import markdown
import xml.etree.ElementTree as ET

In [2]:
def parse_mdx_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        mdx_content = file.read()

    mdx = frontmatter.loads(mdx_content)
    metadata = mdx.metadata
    markdown_content = mdx.content

    # html_content = markdown.markdown(markdown_content, extensions=['markdownx'])

    return metadata, markdown_content


In [3]:
file_path = '../dapi/lib/ai_agent/prompts/prompt_files/mixed_companies_prompt_template.prompt'

In [4]:
!ls ../dapi/lib/ai_agent/prompts/prompt_files 

mixed_companies_prompt_template.prompt mixed_people_prompt_template.prompt


In [5]:
metadata, markdown = parse_mdx_file(file_path)

In [6]:
metadata

{'name': 'Name of the prompt',
 'description': 'Given a subject line, generate email body',
 'id': 'mixed_companies_prompt_template',
 'model': 'gpt-3.5-turbo',
 'temperature': 0.7,
 'max_tokens': -1,
 'top_p': 1.0,
 'presence_penalty': 0.0,
 'frequency_penalty': 0.0,
 'provider': 'openai',
 'endpoint': 'chat',
 'variables': ['DETAILS', 'DERIVED_COLUMN_DETAILS', 'QUERY', 'ANSWER_FORMAT'],
 'tools': [],
 'train_data': '',
 'test_data': '',
 'val_data': ''}

In [7]:
markdown

'<content>\n  <system>\n    You are a helpful analyst who can read the given information and answer the query in a logical way.\n    Your response should be in JSON format and answer should follow the instructions given inside "answerFormat" tag\n  </system>\n  <user>\n    <instructions>\n      Below mentioned are the details of a company inside "companyDetails" tag.\n      Read the details carefully like a professional and understand the properties\n      of the company.\n      Based on the company details answer the question mentioned within "query" tag.\n      Your answer should follow the instructions provided inside "answerFormat" tag\n    </instructions>\n    <companyDetails>\n      {{DETAILS}}\n      {{DERIVED_COLUMN_DETAILS}}\n    </companyDetails>\n    <query>{{QUERY}}</query>\n    <answerFormat>\n      {{ANSWER_FORMAT}}.\n      Do not provide answer inside xml tags\n      Be precise and concise and follow the instructions.\n      Answer should be a single key value paire wher

In [8]:
def _extract_string_from_xml(content, tag, as_string=False):
    root = ET.fromstring(content)
    elements = root.findall(".//" + tag)
    text = None
    if as_string:
        text = [
            ET.tostring(element, encoding="unicode", method="xml")
            for element in elements
        ]
    else:
        text = [element.text for element in elements]
    return " ".join(text)

In [9]:
_extract_string_from_xml(markdown, "user")

'\n    '

In [10]:
_extract_string_from_xml(markdown, "shankar") # there is no shankar tag

''

In [11]:
_extract_string_from_xml(markdown, "shankar", as_string=True) 

''

In [12]:
_extract_string_from_xml(markdown, "user", as_string=True) 

'<user>\n    <instructions>\n      Below mentioned are the details of a company inside "companyDetails" tag.\n      Read the details carefully like a professional and understand the properties\n      of the company.\n      Based on the company details answer the question mentioned within "query" tag.\n      Your answer should follow the instructions provided inside "answerFormat" tag\n    </instructions>\n    <companyDetails>\n      {{DETAILS}}\n      {{DERIVED_COLUMN_DETAILS}}\n    </companyDetails>\n    <query>{{QUERY}}</query>\n    <answerFormat>\n      {{ANSWER_FORMAT}}.\n      Do not provide answer inside xml tags\n      Be precise and concise and follow the instructions.\n      Answer should be a single key value paire where the key is \'answer\'\n    </answerFormat>\n  </user>\n'